In [ ]:
!pip -q install sentence-transformers


In [ ]:
!nvidia-smi


Thu Jan 15 07:11:11 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!ls

dev.json  sample_data  test.json  train.json


In [ ]:
!pip -q install -U sentence-transformers scipy scikit-learn

import json
import numpy as np
from scipy.stats import spearmanr
from collections import defaultdict

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from sentence_transformers import SentenceTransformer


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

TRAIN_PATH = "train.json"
DEV_PATH = "dev.json"
TEST_PATH = "test.json"

class STDM(nn.Module):
    def __init__(self, input_dim, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze(-1)

def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def build_story_text(s):
    ending = s.get("ending", "").strip()
    if ending == "":
        ending = "[NO ENDING]"
    return f"{s['precontext'].strip()} {s['sentence'].strip()} {ending}"

def build_sense_text(s):
    ex = s.get("example_sentence", "").strip()
    if ex:
        return f"{s['judged_meaning'].strip()}. {ex}"
    return s["judged_meaning"].strip()

def accuracy_within_sd(preds, samples):
    correct = 0
    for p, s in zip(preds, samples):
        gold = float(s["average"])
        sd = float(s["stdev"])
        if abs(p - gold) <= max(1.0, sd):
            correct += 1
    return correct / len(samples)

train_data = load_json(TRAIN_PATH)
dev_data = load_json(DEV_PATH)
test_data = load_json(TEST_PATH)

train_samples = list(train_data.values())
dev_samples = list(dev_data.values())
test_samples = list(test_data.values())

encoder = SentenceTransformer(
    "sentence-transformers/all-MiniLM-L6-v2",
    device=DEVICE
)

hidden = encoder.get_sentence_embedding_dimension()
input_dim = 4 * hidden
stdm = STDM(input_dim).to(DEVICE)


def interaction(u, v):
    return torch.cat(
        [u, v, torch.abs(u - v), u * v],
        dim=1
    )


@torch.no_grad()
def evaluate(base_model, stdm, samples):
    base_model.eval()
    stdm.eval()

    stories = [build_story_text(s) for s in samples]
    senses = [build_sense_text(s) for s in samples]

    u = base_model.encode(stories, convert_to_tensor=True)
    v = base_model.encode(senses,  convert_to_tensor=True)

    x = interaction(u, v)
    preds = stdm(x).cpu().numpy()

    gold = np.array([float(s["average"]) for s in samples])

    sp = spearmanr(preds, gold).correlation
    acc = accuracy_within_sd(preds, samples)
    return float(sp), float(acc)


optimizer = torch.optim.AdamW(
    list(encoder.parameters()) + list(stdm.parameters()),
    lr=1e-5
)

mse = nn.MSELoss()


def train_epoch(base_model, stdm, samples, batch_size=16):
    base_model.train()
    stdm.train()

    total_loss = 0.0

    for i in range(0, len(samples), batch_size):
        batch = samples[i:i+batch_size]

        stories = [build_story_text(s) for s in batch]
        senses = [build_sense_text(s) for s in batch]
        targets = torch.tensor(
            [float(s["average"]) for s in batch],
            dtype=torch.float,
            device=DEVICE
        )

        optimizer.zero_grad()

        u = base_model.encode(stories, convert_to_tensor=True)
        v = base_model.encode(senses,  convert_to_tensor=True)

        x = interaction(u, v)
        preds = stdm(x)

        loss = mse(preds, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / (len(samples) // batch_size)

best_val = -1e9

for epoch in range(20):
    loss = train_epoch(encoder, stdm, train_samples)
    sp, acc = evaluate(encoder, stdm, dev_samples)

    if sp > best_val:
        best_val = sp
        torch.save(
            {
                "encoder": encoder.state_dict(),
                "stdm": stdm.state_dict()
            },
            "best_model.pt"
        )

    print(f"Epoch {epoch+1:02d} | Loss {loss:.4f} | Val Spearman {sp:.4f} | Acc@SD {acc:.4f}")

checkpoint = torch.load("best_model.pt", map_location=DEVICE)
encoder.load_state_dict(checkpoint["encoder"])
stdm.load_state_dict(checkpoint["stdm"])

encoder.eval()
stdm.eval()

stories = [build_story_text(s) for s in test_samples]
senses  = [build_sense_text(s) for s in test_samples]

with torch.no_grad():
    u = encoder.encode(stories, convert_to_tensor=True)
    v = encoder.encode(senses,  convert_to_tensor=True)

    preds = stdm(interaction(u, v))
    preds = torch.clamp(preds, 1.0, 5.0)

preds = preds.cpu().numpy()

preds_int = np.rint(preds).astype(int)
preds_int = np.clip(preds_int, 1, 5)

with open("predictions.jsonl", "w", encoding="utf-8") as f:
    for i, p in enumerate(preds_int):
        f.write(
            json.dumps({"id": str(i), "prediction": int(p)}) + "\n"
        )

print("Saved test_predictions.jsonl")
print("Prediction std:", np.std(preds))
print("Unique predictions:", np.unique(preds_int))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 47.1 MB/s eta 0:00:00
Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Epoch 01 | Loss 4.0557 | Val Spearman 0.0263 | Acc@SD 0.5357
Epoch 02 | Loss 1.6007 | Val Spearman 0.0248 | Acc@SD 0.5561
Epoch 03 | Loss 1.5048 | Val Spearman 0.0376 | Acc@SD 0.5663
Epoch 04 | Loss 1.4543 | Val Spearman 0.0544 | Acc@SD 0.5646
Epoch 05 | Loss 1.4211 | Val Spearman 0.0686 | Acc@SD 0.5697
Epoch 06 | Loss 1.3762 | Val Spearman 0.0808 | Acc@SD 0.5697
Epoch 07 | Loss 1.3434 | Val Spearman 0.0954 | Acc@SD 0.5629
Epoch 08 | Loss 1.3161 | Val Spearman 0.1073 | Acc@SD 0.5663
Epoch 09 | Loss 1.2870 | Val Spearman 0.1166 | Acc@SD 0.5629
Epoch 10 | Loss 1.2514 | Val Spearman 0.1265 | Acc@SD 0.5578
Epoch 11 | Loss 1.2262 | Val Spearman 0.1296 | Acc@SD 0.5578
Epoch 12 | Loss 1.2101 | Val Spearman 0.1330 | Acc@SD 0.5578
Epoch 13 | Loss 1.1724 | Val Spearman 0.1385 | Acc@SD 0.5527
Epoch 14 | Loss 1.1369 | Val Spearman 0.1419 | Acc@SD 0.5578
Epoch 15 | Loss 1.1008 | Val Spearman 0.1446 | Acc@SD 0.5561
Epoch 16 | Loss 1.0720 | Val Spearman 0.1467 | Acc@SD 0.5578
Epoch 17 | Loss 1.0401 |